# Вводная часть

## Импорт библиотек

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
# Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

## Функция загрузки данных

In [27]:
def req_to_direct(URL, body, headers):   
    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(URL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет создан успешно")
                format(u(req.text))
                return(req.text)
                # return(req.content) если вдруг хотим в байтах выгрузить и преобразовывать
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break

# Запрос к директу

## Токен доступа

In [32]:
# токен доступа к директу
ACCESS_TOKEN = "ВАШ ТОКЕН"

ACCESS_TOKEN_EZ = "ВАШ ТОКЕН"

## Параметры запроса

In [39]:
# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

#адрес сервиса ОТЧЕТОВ ПЕСОЧНИЦА
ReportsURL_SANDBOX = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN_EZ

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
clientLogin = 'evgeny.zhmuidetsky'

#какие столбцы подгрузить
# для дашборда необходимы дата, тип рекламной сети, показы, клики, стоимость, количество конверсий
stolbec = ["Date", "AdNetworkType", "Impressions", "Clicks", "Cost", "Conversions"]


#даты отчета
StartDate = "2022-06-20"
FinishDate = "2023-02-12"

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
           "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете, если не сделать, то расходы будут в миллионах
            "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           # "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
            #"skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
            "skipReportSummary": "true"
           }

# Создание тела запроса
body = {
    "params": {
        # критерии выбора, тут могут быть фильтры, логические операторы, 
        "SelectionCriteria": {
            "DateFrom": StartDate,
            "DateTo": FinishDate
                             },
        
        "FieldNames": stolbec,
        "ReportName": u("Отчет 98"),
        "ReportType": "CUSTOM_REPORT",   # тип отчета
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",  # обязательно
        "IncludeVAT": "YES",
        "IncludeDiscount": "NO"
              }
       }

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)


## Запрос к директу

In [40]:
# запрос к данным директа
result = req_to_direct(ReportsURL, body, headers)

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 7650561725448280190
Отчет создан успешно


In [41]:
result

'"Отчет 98 (2022-06-20 - 2023-02-12)"\nDate\tAdNetworkType\tImpressions\tClicks\tCost\tConversions\n2022-06-20\tAD_NETWORK\t491\t3\t33.84\t--\n2022-06-20\tSEARCH\t3\t0\t0.00\t--\n2022-06-21\tAD_NETWORK\t816\t3\t37.60\t--\n2022-06-21\tSEARCH\t7\t0\t0.00\t--\n2022-06-22\tAD_NETWORK\t1475\t2\t14.92\t--\n2022-06-22\tSEARCH\t7\t1\t146.24\t--\n2022-06-23\tAD_NETWORK\t1627\t11\t118.68\t--\n2022-06-23\tSEARCH\t9\t1\t116.42\t--\n2022-06-24\tAD_NETWORK\t967\t4\t69.08\t--\n2022-06-24\tSEARCH\t6\t0\t0.00\t--\n2022-06-25\tAD_NETWORK\t558\t3\t33.17\t--\n2022-06-25\tSEARCH\t21\t2\t186.27\t2\n2022-06-26\tAD_NETWORK\t726\t3\t17.71\t--\n2022-06-26\tSEARCH\t13\t1\t16.16\t--\n2022-06-27\tAD_NETWORK\t4155\t39\t436.61\t--\n2022-06-27\tSEARCH\t61\t6\t552.35\t1\n2022-06-28\tAD_NETWORK\t9828\t69\t898.26\t--\n2022-06-28\tSEARCH\t75\t8\t515.45\t--\n2022-06-29\tAD_NETWORK\t6678\t57\t650.84\t1\n2022-06-29\tSEARCH\t70\t8\t490.36\t--\n2022-06-30\tAD_NETWORK\t8323\t62\t984.40\t1\n2022-06-30\tSEARCH\t102\t11\t998.34\t

## Преобразование ответа в датафрейм

In [42]:
# Преобразование ответа в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(result)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

Date AdNetworkType  Impressions  Clicks   Cost Conversions
0  2022-06-20    AD_NETWORK          491       3  33.84          --
1  2022-06-20        SEARCH            3       0   0.00          --
2  2022-06-21    AD_NETWORK          816       3  37.60          --
3  2022-06-21        SEARCH            7       0   0.00          --
4  2022-06-22    AD_NETWORK         1475       2  14.92          --

## Преобразование форматов

In [43]:
# Столбец даты переводить в формат даты нельзя, это мешает записи в файл
# столбец конверсий в формат целых чисел
df['Conversions'] = df['Conversions'].replace('--', '0')
df['Conversions'] = df['Conversions'].astype('int')

## Подсчет CTR,  средней стоимости клика и конверсии

In [59]:
df['Aver_click_cost'] = round(df['Cost'] / df['Clicks'] , 2)
# если стоимости нет заполним ячейки нулем
df['Aver_click_cost'] = df['Aver_click_cost'].fillna(0)
df['CTR'] = round((df['Clicks'] / df['Impressions']) *100 , 2)

# посчитаем стоимость целевого действия, если его  не было то оно = 0
df.loc[df['Conversions'] == 0, 'Aver_target_cost'] = 0
df.loc[df['Conversions'] != 0, 'Aver_target_cost'] = round(df['Cost'] / df['Conversions'] , 2)
#df['Aver_terget_cost'] = round(df['Cost'] / df['Conversions'] , 2)

# Загрузка файла на Гугл диск

## Загрузка библиотек

In [45]:
# Подключаем библиотеки
import httplib2 
import apiclient.discovery
from oauth2client.service_account import ServiceAccountCredentials

from __future__ import print_function
import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
import pprint
import io

pp = pprint.PrettyPrinter(indent=4)

# библиотека работы с Гугл шитс
# https://docs.gspread.org/en/latest/user-guide.html#opening-a-spreadsheet
import gspread
import gspread_dataframe as gd

## Файл с ключом доступа

In [46]:
# Ссылка на файл с закрытым ключом
CREDENTIALS_FILE = r'C:\Users\ezhmui04\Documents\Ключ доступа к GoogleCloud\directdashboard-734cbb03b84d.json' 
# создание SCOPE только для драйва
SCOPES_drive = ['https://www.googleapis.com/auth/drive']

In [47]:
credentials_drive = service_account.Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES_drive)
gc = gspread.service_account(CREDENTIALS_FILE)

## Создаем файл Data_for_dash

In [48]:
# Читаем ключи из файла
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive'])

httpAuth = credentials.authorize(httplib2.Http()) # Авторизуемся в системе
service = apiclient.discovery.build('sheets', 'v4', http = httpAuth) # Выбираем работу с таблицами и 4 версию API

spreadsheet = service.spreadsheets().create(body = {
    'properties': {'title': 'Data_for_dash', 'locale': 'ru_RU'},
    'sheets': [{'properties': {'sheetType': 'GRID',
                               'sheetId': 0,
                               'title': 'List1',
                               'gridProperties': {'rowCount': 10000, 'columnCount': 15}}}]
}).execute()
spreadsheetId = spreadsheet['spreadsheetId'] # сохраняем идентификатор файла
print('https://docs.google.com/spreadsheets/d/' + spreadsheetId)

https://docs.google.com/spreadsheets/d/193gD3jEMjezNXZDLg93RsDfV-UTbshsyNWODVXPQJNE


## разрешение на чтение файла

In [63]:
# выдача разрешения на чтение файла 
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'rostov78@mail.ru'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

# role can be also 'writer'

In [65]:
# выдача разрешения на чтение файла 
driveService = apiclient.discovery.build('drive', 'v3', http = httpAuth) # Выбираем работу с Google Drive и 3 версию API
access = driveService.permissions().create(
    fileId = spreadsheetId,
    body = {'type': 'user', 'role': 'writer', 'emailAddress': 'zhmuidetsky@yandex.ru'},  # Открываем доступ на редактирование
    fields = 'id'
).execute()

## Запись датафрейма в файл

In [50]:
df['Date'] = df['Date'].astype(str)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              451 non-null    object 
 1   AdNetworkType     451 non-null    object 
 2   Impressions       451 non-null    int64  
 3   Clicks            451 non-null    int64  
 4   Cost              451 non-null    float64
 5   Conversions       451 non-null    int32  
 6   Aver_click_cost   397 non-null    float64
 7   CTR               451 non-null    float64
 8   Aver_target_cost  451 non-null    float64
dtypes: float64(4), int32(1), int64(2), object(2)
memory usage: 30.1+ KB


In [61]:
# Open a spreadsheet by title
sh = gc.open("Data_for_dash")
wk = sh.sheet1

# обязательно указать raw=False - что бы не было лишнего апострофа перед датой
params = {'valueInputOption': 'USER_ENTERED'}
# запись в файл датафрейма целиком
wk.update([df.columns.values.tolist()] + df.values.tolist(), raw=False)

{'spreadsheetId': '193gD3jEMjezNXZDLg93RsDfV-UTbshsyNWODVXPQJNE',
 'updatedRange': 'List1!A1:I452',
 'updatedRows': 452,
 'updatedColumns': 9,
 'updatedCells': 4068}